In [10]:
# contract always success
!echo -e "int main() {\n\
    return 0;\n\
}" > contracts/always_success.c

In [11]:
!riscv64-unknown-elf-gcc -O3 -mcmodel=medlow -Wall -Werror -Wno-nonnull-compare -Wno-unused-function -Wl,-static -fdata-sections -ffunction-sections -Wl,--gc-sections -Wl,-s -o build/always_success contracts/always_success.c

In [10]:
# connect rpc wrapper
from tinyrpc.protocols.jsonrpc import JSONRPCProtocol
from tinyrpc.transports.http import HttpPostClientTransport
from tinyrpc import RPCClient

rpc_client = RPCClient(
    JSONRPCProtocol(),
    HttpPostClientTransport('http://127.0.0.1:8999/'))

remote_server = rpc_client.get_proxy()

remote_server.blockNumber()

12455

In [13]:
# index 0 user for delpoy contract
remote_server.getHDUserInfo(0)

{'privkey': '0xbb918557dad030f8c16d2cebb3da66d380e11ede53214d6e8097d4967bed9d3c',
 'pubkey': '0x025575c6dc037c40b2f7e5cccdb2f071135fe60dfbe1c85bf5a6052734bab7eaaa',
 'blake160': '0x32d0281300319169a9c6f01f1bd6ee566089bdfe',
 'address': 'ckt1qyqr95pgzvqrrytf48r0q8cm6mh9vcyfhhlq4hj46t'}

In [14]:
!pwd

/home/rink/work/github/ckb-notebook


In [2]:
# send some capacity to account index 0 
remote_server.deployContract("0xbb918557dad030f8c16d2cebb3da66d380e11ede53214d6e8097d4967bed9d3c" ,"/home/rink/work/github/ckb-notebook/build/always_success")

{'name': 'always_success',
 'elf_path': '/home/rink/work/github/ckb-notebook/build/always_success',
 'code_hash': '0x0274eb897aef04482f737d3fbee9c5983e510622c6cecd78f545e433ae6e70f0',
 'hash_type': 'data',
 'tx_hash': '0x04b4f2a16333adabe35ab3fdcf7fc1f2b57c1177983cd3fae468763ff6217e30',
 'index': '0x0',
 'dep_type': 'code'}

In [3]:
# index 1 user for test
# send some capacity to account index 1
remote_server.getHDUserInfo(1)

{'privkey': '0x2f3ef8c71d1eea37ccb923d3391ceff6fbf9bc53a797bfd66cf4430e031ba27f',
 'pubkey': '0x025d75e3426b74fc6c832dcb3469ae9374be6173f1940cfb14597b5a6c52c16931',
 'blake160': '0xfd03fd824f9b777f3671ab776bd694cc76e3c09d',
 'address': 'ckt1qyq06qlasf8ekamlxec6kamt662vcahrczwsfucj4q'}

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from pyDatalog import pyDatalog
from sqlalchemy import Column, Integer, String, Text

engine = create_engine('sqlite:////home/rink/work/github/ckb-rpc-wrapper/sync.db?check_same_thread=False', echo=False)
# define a base class with SQLAlchemy and pyDatalog capabilities
Base = declarative_base(cls=pyDatalog.Mixin, metaclass=pyDatalog.sqlMetaMixin)
# open a session on a database, then associate it to the Base class
Session = sessionmaker(bind=engine)
session = Session()
Base.session = session
Base.metadata.bind = engine

class Info(Base):
    __tablename__ = 'info'
    __table_args__ = {'autoload':True} # autoload the model
    
    def __repr__(self):
        return "<Info(tip_height='%s')>" % (
                   self.tip_height)

class LiveCells(Base):
    __tablename__ = 'livecells'
    __table_args__ = {'autoload':True} # autoload the model
    
    def __repr__(self):
        return "<LiveCells(id = '%s', tx_hash='%s', cell_index='%s', capacity='%s', lock_code_hash='%s', lock_args='%s', lock_hash_type='%s', type_code_hash='%s', type_args='%s', type_hash_type='%s', height='%s')>" % (
                   self.id, self.tx_hash, self.cell_index, self.capacity, self.lock_code_hash, self.lock_args, self.lock_hash_type, self.type_code_hash, self.type_args, self.type_hash_type, self.height)

X = pyDatalog.Variable()
LiveCells.height[X] == 500
print(X)

pyDatalog.create_terms('link, can_reach')
Y = pyDatalog.Variable()
not_enough(X) <= LiveCells.capacity[Y] > 203850000000
query_live_cells(AMOUNT,[C]) <= link(X,Z) & can_reach(Z,Y) & (X!=Y)

print (query_live_cells(400000000000,Y))

[<LiveCells(id = '489', tx_hash='0x93b1d56938c20e26689bab69f46f45b522018c0f27a3d0fab9382060b98a5fe9', cell_index='0', capacity='203264251627', lock_code_hash='0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', lock_args='0x4a88cef22e4e71c48c40da51c1d6bd16daa97aa7', lock_hash_type='type', type_code_hash='', type_args='', type_hash_type='', height='500')>]


In [15]:
def example_tx():
    tx = {"version":"0x0",
          "cell_deps":[
              {"out_point":
                   {"tx_hash":"0xe02c8a20a64a336cac1e477b9847888251a30a9bfe2d9050ca7ab59f507b15c7",
                    "index":"0x0"},
               "dep_type":
                   "dep_group"}
          ],
          "header_deps":[],
          "inputs":[
              {"previous_output":
                   {"tx_hash":"0xb364c95ba1170b640d72c514f05389c6b37580568e82afb81b28e06935b6309e",
                    "index":"0x0"},
               "since":"0x0"}
          ],
          "outputs":[
              {"capacity":"0x2cb417800",
               "lock":
                   {"code_hash":"0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8",
                    "args":"0xe73ac97da522fbe645811dc5dbfab830d3ca550f",
                    "hash_type":"type"}
              }
          ],
          "outputs_data":["0x"],
          "witnesses":["0xc50477e52d648990415dcb6a0adb9c21df1413e3d738d9001d22b5882398406a01e35a615a26e9713fddfd8840590f756880fdabc4a88ca0c2086a6f074fd9ee00"]
         }
    secp_info = remote_server.systemScript()
    cell_dep = {"outpoint": {"tx_hash": secp_info["tx_hash"], "index": secp_info["index"]}, "dep_type": secp_info["dep_type"]}
    tx["cell_deps"] = [cell_dep]
    return tx
    
tx = example_tx()
print(tx)

import json
tx_json = json.dumps(tx)
print(tx_json)

{'version': '0x0', 'cell_deps': [{'outpoint': {'tx_hash': '0x62b47d45ee689fb852af9002c318dcea1aeb9a4632991429fc3ec38676e356e3', 'index': '0x0'}, 'dep_type': 'dep_group'}], 'header_deps': [], 'inputs': [{'previous_output': {'tx_hash': '0xb364c95ba1170b640d72c514f05389c6b37580568e82afb81b28e06935b6309e', 'index': '0x0'}, 'since': '0x0'}], 'outputs': [{'capacity': '0x2cb417800', 'lock': {'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', 'args': '0xe73ac97da522fbe645811dc5dbfab830d3ca550f', 'hash_type': 'type'}}], 'outputs_data': ['0x'], 'witnesses': ['0xc50477e52d648990415dcb6a0adb9c21df1413e3d738d9001d22b5882398406a01e35a615a26e9713fddfd8840590f756880fdabc4a88ca0c2086a6f074fd9ee00']}
{"version": "0x0", "cell_deps": [{"outpoint": {"tx_hash": "0x62b47d45ee689fb852af9002c318dcea1aeb9a4632991429fc3ec38676e356e3", "index": "0x0"}, "dep_type": "dep_group"}], "header_deps": [], "inputs": [{"previous_output": {"tx_hash": "0xb364c95ba1170b640d72c514f05389c6b375805